# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
cities_df = pd.read_csv("output_data/cityweather_data.csv")
cities_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ilulissat,69.2167,-51.1000,39.22,60,3,0.00,GL,1622686274
1,1,Bluff,-46.6000,168.3333,49.80,76,76,7.38,NZ,1622686274
2,2,Indianola,41.3581,-93.5574,71.85,68,5,2.84,US,1622686275
3,3,Chicama,-7.8447,-79.1469,65.88,82,100,5.86,PE,1622686275
4,4,Hilo,19.7297,-155.0900,82.13,66,20,9.22,US,1622686275


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [16]:
 locations = cities_df[["Lat", "Lng"]]
weights = cities_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# filter out high max temps
hotel_df = cities_df.loc[(cities_df["Max Temp"]< 90) & (cities_df["Humidity"]< 30) & (cities_df["Wind Speed"]< 5)]

hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df["Hotel Name"] = ""
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name


In [9]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)  
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    if len(cities_lat_lng["results"]) > 0:
        hotel_df.loc[index, "Hotel Name"] = cities_lat_lng["results"][0]["name"]
    
    else:
        hotel_df.loc[index, "Hotel Name"] = None


# Visualize to confirm lat lng appear
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name


In [13]:
 # save as csv
hotel_df.to_csv('/Users/makba/Documents/0 Data Science Bootcamp Rice University/HomeWorks/Homework 6 API Challenge/API-Challenge/cities_hotels.csv', index = False)

# open and read 
hotel_df = pd.read_csv('/Users/makba/Documents/0 Data Science Bootcamp Rice University/HomeWorks/Homework 6 API Challenge/API-Challenge/cities_hotels.csv')
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
 # Add marker layer ontop of heat map
records = hotel_df[["Lat","Lng"]].to_records(index=False)
coordinates = list(records)
print(coordinates)

markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

[]


Figure(layout=FigureLayout(height='420px'))